In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1638414109035_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,") \
    .appName("ProcessingBronzeToSilver") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def generate_access_keys():
    aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
    aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
    return aws_access_key_id, aws_secret_key

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
aws_access_key_id, aws_secret_key = generate_access_keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", aws_access_key_id)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", aws_secret_key)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def generate_datalake_layer(layer: str) -> str:
    return f"university-datalake-{layer}"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def generate_path(layer: str, table: str, without_partition=None) -> str:
    if without_partition == True:
        return f"s3a://{layer}/table={table}/{table}.parquet"
    return f"s3a://{layer}/*/*/*/table={table}/{table}.parquet"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
bronze = generate_datalake_layer(layer="bronze")
silver = generate_datalake_layer(layer="silver")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_alunos = spark.read.format('parquet').load(generate_path(layer=bronze, table="alunos"))
df_cursos = spark.read.format('parquet').load(generate_path(layer=bronze, table="cursos"))
df_departamentos = spark.read.format('parquet').load(generate_path(layer=bronze, table="departamentos"))
df_disciplinas = spark.read.format('parquet').load(generate_path(layer=bronze, table="disciplinas"))
df_matriculas = spark.read.format('parquet').load(generate_path(layer=bronze, table="matriculas"))
df_matrizes_cursos = spark.read.format('parquet').load(generate_path(layer=bronze, table="matrizes_cursos"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_alunos = df_alunos.select(col('mat_alu').cast(IntegerType()), 
                             col('nome'), col('dat_entrada'), 
                             col('cod_curso').cast(IntegerType()), col('cotista'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def get_last_element(l):
    return l[-1]

get_last_element_udf = udf(get_last_element)

df_cursos = df_cursos.withColumn('turno', get_last_element_udf(split(df_cursos['nom_curso'], ' ')))
df_cursos = df_cursos.withColumn('nome_curso', split(df_cursos['nom_curso'], '-').getItem(0))

df_cursos = df_cursos.select(col('cod_curso').cast(IntegerType()),
                             col('nome_curso'), col('cod_dpto').cast(IntegerType()),
                             col('turno'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_departamentos = df_departamentos.select(col('cod_dpto').cast(IntegerType()),
                                           col('nome_dpto'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_disciplinas = df_disciplinas.select(col('cod_disc').cast(IntegerType()),
                                           col('nome_disc'), col('carga_horaria').cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_matriculas = df_matriculas.select(col('semestre').cast(IntegerType()),
                                     col('mat_alu').cast(IntegerType()), 
                                     col('cod_disc').cast(IntegerType()),
                                     col('nota').cast(FloatType()),
                                     col('faltas').cast(IntegerType()),
                                     col('status'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_matrizes_cursos = df_matrizes_cursos.select(col('cod_curso').cast(IntegerType()),
                                     col('cod_disc').cast(IntegerType()), 
                                     col('periodo').cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_alunos.write.format('parquet') \
         .mode("overwrite") \
         .option('path', generate_path(layer=silver, table="alunos", without_partition=True)).save()
    
df_cursos.write.format('parquet') \
         .mode("overwrite") \
         .option('path', generate_path(layer=silver, table="cursos", without_partition=True)).save()
    
df_departamentos.write.format('parquet') \
         .mode("overwrite") \
         .option('path', generate_path(layer=silver, table="departamentos", without_partition=True)).save()
    
df_disciplinas.write.format('parquet') \
         .mode("overwrite") \
         .option('path', generate_path(layer=silver, table="disciplinas", without_partition=True)).save()
    
df_matriculas.write.format('parquet') \
         .mode("overwrite") \
         .option('path', generate_path(layer=silver, table="matriculas", without_partition=True)).save()
    
df_matrizes_cursos.write.format('parquet') \
         .mode("overwrite") \
         .option('path', generate_path(layer=silver, table="matrizes_cursos", without_partition=True)).save()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
return "The processing was finished and the data it's on the silver layer!"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The processing was finished and the data it's on the silver layer!